In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

#import for colab
from google.colab import drive
drive.mount('/content/drive')

import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
src_img_dir = './drive/MyDrive/desc_patch_extract/aachen_train/'
images = os.listdir(src_img_dir)

out_img_dir = './drive/MyDrive/desc_patch_extract/aachen_patches/'

In [ ]:
counter = 0

In [ ]:
for img_file in images:
    counter += 1
    if counter == 201:
      break

    img = cv2.imread(src_img_dir + img_file)
    
    sift = cv2.SIFT_create(0, 1, 0.03)
    kpt = sift.detect(img, None)
    #img = cv2.drawKeypoints(img_gray, kpt, img, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    for i in range(len(kpt)):
        '''num_rows, num_cols = img.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((kpt[i].pt[0], kpt[i].pt[1]), kpt[i].angle, 1)
        transformed = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
        
        patch = transformed[int(kpt[i].pt[1] - kpt[i].size / 2) : int(kpt[i].pt[1] + kpt[i].size / 2), int(kpt[i].pt[0] - kpt[i].size / 2) : int(kpt[i].pt[0] + kpt[i].size / 2)]
        patch = cv2.resize(patch, (16, 16))'''
        
        rect = ((kpt[i].pt[0], kpt[i].pt[1]), (kpt[i].size, kpt[i].size), kpt[i].angle)
        box = cv2.boxPoints(rect)

        dst_pts = np.array([[0, kpt[i].size], [0, 0], [kpt[i].size, 0], [kpt[i].size, kpt[i].size]], dtype="float32")

        M = cv2.getPerspectiveTransform(box, dst_pts)
        warped = cv2.warpPerspective(img, M, (int(kpt[i].size), int(kpt[i].size)))
        warped = cv2.resize(warped, (16, 16))
        
        cv2.imwrite(out_img_dir + img_file[:-4] + '_' + str(i) + img_file[-4:], warped)

KeyboardInterrupt: ignored

In [ ]:
img_name = './localize/r-1294886518.812359-3922203127.png'
img = cv2.imread(img_name)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

i = 221
num_rows, num_cols = img.shape[:2]

sift = cv2.SIFT_create(0, 1, 0.03)
kpt = sift.detect(img, None)

img = cv2.drawKeypoints(img_gray, kpt, img, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

#old version
num_rows, num_cols = img.shape[:2]
rotation_matrix = cv2.getRotationMatrix2D((kpt[i].pt[0], kpt[i].pt[1]), kpt[i].angle, 1)
transformed = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
        
patch = transformed[int(kpt[i].pt[1] - kpt[i].size / 2) : int(kpt[i].pt[1] + kpt[i].size / 2), int(kpt[i].pt[0] - kpt[i].size / 2) : int(kpt[i].pt[0] + kpt[i].size / 2)]

#new version
rect = ((kpt[i].pt[0], kpt[i].pt[1]), (kpt[i].size, kpt[i].size), kpt[i].angle)
box = cv2.boxPoints(rect)

dst_pts = np.array([[0, kpt[i].size],
                    [0, 0],
                    [kpt[i].size, 0],
                    [kpt[i].size, kpt[i].size]], dtype="float32")

M = cv2.getPerspectiveTransform(box, dst_pts)
warped = cv2.warpPerspective(img, M, (int(kpt[i].size), int(kpt[i].size)))

fx, plots = plt.subplots(1, 2, figsize=(20,10))
plots[0].imshow(patch)
plots[1].imshow(warped)